In [ ]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join

In [ ]:
fac_index = list(range(1,41))

In [ ]:
for ind in fac_index: 
    mypath = r'D:\AltFemp\Facility\fac'+ str(ind)
    files = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [ ]:
# load the center data 

In [ ]:
df = pd.read_excel(r'D:\AltFemp\raw_data\cneter.xlsx')

In [ ]:
def calsigma(ws,x):
    # Assumptions: 1. Moderate Incoming Radiation
    #              2. Open Country
    # ws -> the wind speed 
    # x -> Downwind distance
    x = np.float32(x)
    if ws<2:
        sigy = 0.22*x*(1+0.0001*x)**(-1/2)
        sigz = 0.2*x
    elif(2<= ws < 5):
        sigy = 0.16*x*(1+0.0001*x)**(-1/2)
        sigz = 0.12*x
    elif(5<= ws < 6):
        sigy = 0.11*x*(1+0.0001*x)**(-1/2)
        sigz = 0.08*x*(1+0.0002*x)**(-1/2)
    else:
        sigy = 0.08*x*(1+0.0001*x)**(-1/2)
        sigz = 0.06*x*(1+0.0015*x)**(-1/2)

    return sigy,sigz

In [ ]:
df.head()

In [ ]:
for ind in fac_index: 
    # loading the mean center coordiates
    print (ind)
    center = df.iloc[ind-1]
    x = center[0]
    y = center[1]
    # loading the facility 
    mypath = r'D:\AltFemp\Facility\fac'+ str(ind)
    files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
    i = 0 
    for f in files:
        # loading the dataset 
        fdf = pd.read_excel(mypath +'\\' + f)
        # removed the lost signal data  
        fdf = fdf.dropna()
        # check the dimension of dataset
        dim = fdf.shape
        # assign the attribute
        pt_x = fdf.x_utm
        pt_y = fdf.y_utm 
        wspd = fdf.true_wspd
        wdir = fdf.true_wdir
        c = fdf.ch4_moleratio
        # zip the attribute 
        data = list(zip(pt_x,pt_y,wspd,wdir,c))
        # create empty list for saving new attributes 
        dist_list = []
        Q_list =[]
        Pang = []
        for d in data:
            # calculate the distance to the mean center
            dist = ((d[0]-x)**2 + (d[1]-y)**2)**(1/2) 
            # calculatet the angle between the point and mean center 
            ang = np.arctan2((d[1]-y),(d[0]-x))*180/np.pi
            # calculate the simmilaritry betwene wind direction and the angle with mean center 
            pang = np.absolute(d[3]/ang)
            # calculate sigy and sigz 
            sigy,sigz = calsigma(d[2],dist)
            # calculate the emission rate 
            Q = d[2]*np.pi*d[4]*sigy*sigz/np.exp(-1)
            # save the calculated values into list 
            dist_list.append(dist)
            Q_list.append(Q)
            Pang.append(pang)
        # add to the dataframe     
        fdf['dist_to_source'] = dist_list
        fdf['emission_rate'] = Q_list
        fdf['wdir_prob'] = Pang

        fdf.to_excel(r'D:\AltFemp\cal_fac\%s'%f)
        i += 1 
        print (i)